#Summarize select link flow results by time of day periods to base scenario and export
##setup and create attribute to store total select link flows
##export the total select link flows to a CSV file
##Initial Script: Ziying Ouyang Feb 2019

In [1]:
import os
import pandas as pd
import math
from IPython.display import display
import numpy as py

modeller = inro.modeller.Modeller()
emmebank = modeller.emmebank

In [2]:
scenario_id = 100
base_scenario = emmebank.scenario(scenario_id)
base_network = base_scenario.get_network()
main_directory = os.path.dirname(os.path.dirname(modeller.desktop.project.path)).replace("\\","/")
export_path = os.path.join(main_directory, "output").replace("\\","/")
file_name = os.path.join(export_path, "loadselk.csv")
import_path = os.path.join(main_directory, "emme_project\Database").replace("\\","/")
file_name_input = os.path.join(import_path, "Sl_query.csv")
suffixes = pd.read_csv(file_name_input,sep=',')
suffixes = suffixes['Query']
id_format = lambda x: str(int(x))

In [3]:
#daily select link volumes by selection set
# the names need to match with the names in the select link result suffix in select link section
# of master run

#suffixes = ["dtwn","uptn","kmsa","svly"]
#suffixes = ["enci"]
total_names = ["@sel_" + suffix 
             for suffix in suffixes]

In [4]:
hwyload_attrs = [("ID1", "@tcov_id"),("length","length")]
# AB daily select link volume by section set
dir_atts = [("AB_sel" + name[5:], name )  
           for name in total_names]

for key, attr in dir_atts:
            hwyload_attrs.append((key, attr))
            hwyload_attrs.append((key.replace("AB_", "BA_"), (attr, "")))  # default for BA on one-way links is blank
#display(hwyload_attrs)

[('ID1', '@tcov_id'),
 ('length', 'length'),
 ('AB_selcarl', '@sel_carl'),
 ('BA_selcarl', ('@sel_carl', '')),
 ('AB_selchul', '@sel_chul'),
 ('BA_selchul', ('@sel_chul', '')),
 ('AB_selcoro', '@sel_coro'),
 ('BA_selcoro', ('@sel_coro', '')),
 ('AB_seldelm', '@sel_delm'),
 ('BA_seldelm', ('@sel_delm', ''))]

In [5]:
auto_mode = base_network.mode("d")
# only the original forward direction links and auto links only
links = [l for l in base_network.links() 
         if l["@tcov_id"] > 0 and 
         (auto_mode in l.modes or (l.reverse_link and auto_mode in l.reverse_link.modes))
        ]
links.sort(key=lambda l: l["@tcov_id"])
with open(file_name, 'w') as fout:
    fout.write(",".join(['"%s"' % x[0] for x in hwyload_attrs]))
    fout.write("\n")
    for link in links:
        key, att = hwyload_attrs[0]  # expected to be the link id
        values = [id_format(link[att])]
        reverse_link = link.reverse_link
        for key, att in hwyload_attrs[1:]:
            if key.startswith("BA"):
                name, default = att
                
                if reverse_link and (abs(link["@tcov_id"]) == abs(reverse_link["@tcov_id"])):
                    values.append(format(reverse_link[name]))
                else:
                    values.append(default)

                #values.append(format(reverse_link[name]) if reverse_link else default)
            elif att.startswith("#"):
                values.append('"%s"' % link[att])
            else:
                values.append(format(link[att]))
        fout.write(",".join(values))
        fout.write("\n")


In [26]:
data = pd.read_csv(file_name,sep=',')
names=[]
for s in suffixes:
    AB_Flow = 'AB_sel' + s
    BA_Flow = 'BA_sel' + s
    total_flow = "Total_Flow_sel" + s
    AB_VMT = "AB_VMT_sel" + s
    BA_VMT = "BA_VMT_sel" + s
    total_VMT = "Total_VMT_sel" + s
    data[total_flow] = data[AB_Flow] + data[BA_Flow ] 
    data[AB_VMT] = data[AB_Flow ] * data['length'] 
    data[BA_VMT] = data[BA_Flow] * data['length'] 
    data[total_VMT] = data[total_flow] * data['length']
    names.extend([AB_Flow,BA_Flow,total_flow,AB_VMT,BA_VMT,total_VMT]) 
colnames = ["ID1", "length"] + names
data = data[colnames]
data.to_csv(file_name, index= False)